In [1]:
#calling all needed libraries

import pandas as pd
import numpy as np
from collections import defaultdict
from rapidfuzz import process, fuzz

In [2]:
#Grabbing data and throwing them into the dictionary call dfs

file_path = r'Data\new data.xlsx'

excel_data = pd.ExcelFile(file_path)

dfs = {}

for sheet_name in excel_data.sheet_names:
    dfs[sheet_name] = pd.read_excel(excel_data, sheet_name=sheet_name, header=0)

for df_name, df in dfs.items():
    if 'Rk' in df.columns:
        df.drop(columns=['Rk'], inplace=True)

In [3]:
#creates a master list of names, account for spelling errors

unique_names = set()

for df in dfs.values():
    if 'name' in df.columns:
        unique_names.update(df['name'].dropna().unique())

unique_names_list = list(unique_names)
master_df = pd.DataFrame(unique_names_list, columns=['name'])

name_mapping = {}
standard_names = []

for name in master_df['name']:
    if standard_names:
        match, score, _ = process.extractOne(name, standard_names, scorer=fuzz.WRatio)
        if score >= 85:
            name_mapping[name] = match
        else:
            standard_names.append(name)
            name_mapping[name] = name
    else:
        standard_names.append(name)
        name_mapping[name] = name

master_df['name'] = master_df['name'].replace(name_mapping)

for key, df in dfs.items():
    if 'name' in df.columns:
        df['name'] = df['name'].replace(name_mapping)

In [4]:
#counts the number of times a person has received 1st, 2nd, 3rd, or 4th

result_counts = defaultdict(lambda: defaultdict(int))

for key, df in dfs.items():
    df.columns = df.columns.str.strip()
    
    name_column = next((col for col in df.columns if 'name' in col.lower()), None)
    if not name_column:
        continue

    rank_columns = [col for col in df.columns if col.startswith('Rk')]
    
    for _, row in df.iterrows():
        name = row[name_column]
        for col in rank_columns:
            if row[col] in ['1st', '2nd', '3rd', '4th']:
                result_counts[name][row[col]] += 1

master_names = master_df['name'].tolist()
filtered_results = {name: result_counts[name] for name in master_names}

for rank in ['1st', '2nd', '3rd', '4th']:
    master_df[rank] = master_df['name'].map(lambda name: filtered_results.get(name, {}).get(rank, 0))

In [5]:
score_columns = ["R1", "R2", "R3", "R4", "R5"]

master_df['Total Accumulated Score'] = 0
master_df['Overall Average Score'] = 0.0
master_df['Count Appearances in DataFrames'] = 0

for col in score_columns:
    master_df[f'Total {col} Score'] = 0.0
    master_df[f'Count {col} Appearances'] = 0
    master_df[f'Average {col}'] = 0.0

for key, df in dfs.items():
    df.columns = df.columns.str.strip()
    
    name_column = next((col for col in df.columns if 'name' in col.lower()), None)
    if not name_column:
        continue

    available_columns = [col for col in score_columns if col in df.columns]
    for col in available_columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    for _, row in df.iterrows():
        name = row[name_column]
        if name in master_df['name'].values:
            master_df.loc[master_df['name'] == name, 'Count Appearances in DataFrames'] += 1

            total_score = row[available_columns].sum()
            average_score = row[available_columns].mean()

            master_df.loc[master_df['name'] == name, 'Total Accumulated Score'] += total_score

            for col in available_columns:
                master_df.loc[master_df['name'] == name, f'Total {col} Score'] += row[col]
                master_df.loc[master_df['name'] == name, f'Count {col} Appearances'] += 1

for col in score_columns:
    master_df[f'Average {col}'] = (
        master_df[f'Total {col} Score'] / master_df[f'Count {col} Appearances']
    ).fillna(0).round(2)

master_df['Overall Average Score'] = (
    master_df['Total Accumulated Score'] / master_df['Count Appearances in DataFrames']
).fillna(0).round(2)

master_df.drop(
    [f'Total {col} Score' for col in score_columns]
    + [f'Count {col} Appearances' for col in score_columns]
    + ['Count Appearances in DataFrames'],
    axis=1,
    inplace=True
)

In [6]:
positions = ["Opening Government", "Opening Opposition", "Closing Government", "Closing Opposition"]
for position in positions:
    master_df[position] = 0

for sheet_name, df in dfs.items():
    if "name" in df.columns and any(col.startswith("side") for col in df.columns):
        side_columns = [col for col in df.columns if col.startswith("side")]
        
        for _, row in df.iterrows():
            participant_name = row["name"]
            
            for side_col in side_columns:
                if side_col in row and row[side_col] in positions:
                    if participant_name in master_df["name"].values:
                        master_df.loc[master_df["name"] == participant_name, row[side_col]] += 1

In [7]:
position_score = defaultdict(lambda: pd.DataFrame(columns=["Side", "Total", "1st", "2nd", "3rd", "4th"]))

sides = ["Opening Government", "Opening Opposition", "Closing Government", "Closing Opposition"]
ranks = ["1st", "2nd", "3rd", "4th"]

for sheet_name, df in dfs.items():
    print(f"Processing sheet: {sheet_name}")
    
    if "name" not in df.columns:
        print(f"Skipping sheet {sheet_name}: No 'name' column")
        continue

    side_columns = [col for col in df.columns if col.startswith("side")]
    rank_columns = [col for col in df.columns if col.startswith("Rk")]

    if not side_columns or not rank_columns:
        print(f"Skipping sheet {sheet_name}: No 'side' or 'Rk' columns")
        continue

    for side_col, rank_col in zip(side_columns, rank_columns):
        for _, row in df.iterrows():
            name = row["name"]

            side = row.get(side_col, None)
            rank = row.get(rank_col, None)

            if pd.notna(side) and pd.notna(rank) and side in sides:
                if side not in position_score[name]["Side"].values:
                    new_row = {"Side": side, "Total": 0, "1st": 0, "2nd": 0, "3rd": 0, "4th": 0}
                    position_score[name] = pd.concat([position_score[name], pd.DataFrame([new_row])], ignore_index=True)
                
                position_score[name].loc[position_score[name]["Side"] == side, "Total"] += 1
                if rank in ranks:
                    position_score[name].loc[position_score[name]["Side"] == side, rank] += 1

position_score = dict(position_score)

Processing sheet: Linfield
Processing sheet: YODL 2
Processing sheet: UCLA IV
Processing sheet: YODL 1
Processing sheet: Princeton
Processing sheet: Links
Skipping sheet Links: No 'name' column


In [8]:
rank_to_points = {"1st": 3, "2nd": 2, "3rd": 1, "4th": 0}

score_columns = ["Opening Government Score", "Opening Opposition Score", "Closing Government Score", "Closing Opposition Score"]
for col in score_columns:
    master_df[col] = 0

for name, df in position_score.items():
    if name not in master_df["name"].values:
        print(f"Skipping {name}: not found in master_df")
        continue

    for side in df["Side"].unique():
        side_data = df[df["Side"] == side].iloc[0]
        total_score = (
            side_data["1st"] * rank_to_points["1st"] +
            side_data["2nd"] * rank_to_points["2nd"] +
            side_data["3rd"] * rank_to_points["3rd"] +
            side_data["4th"] * rank_to_points["4th"]
        )
        
        score_col = f"{side} Score"
        if score_col in master_df.columns:
            master_df.loc[master_df["name"] == name, score_col] = total_score

In [9]:
average_columns = [
    "Opening Government Average", 
    "Opening Opposition Average", 
    "Closing Government Average", 
    "Closing Opposition Average"
]
for col in average_columns:
    master_df[col] = 0.0

for name, df in position_score.items():
    if name not in master_df["name"].values:
        print(f"Skipping {name}: not found in master_df")
        continue

    for side in df["Side"].unique():
        side_data = df[df["Side"] == side].iloc[0]
        total_score = master_df.loc[master_df["name"] == name, f"{side} Score"].values[0]
        total_count = side_data["Total"]

        average_score = total_score / total_count if total_count > 0 else 0

        average_col = f"{side} Average"
        if average_col in master_df.columns:
            master_df.loc[master_df["name"] == name, average_col] = average_score